# Amazon Uk의 신발 제품 리뷰 데이터 감정 분석(Sentiment Analysis)

========================================================================

본 과제에서는 Amazon uk(영국)쇼핑몰의 신발 제품 판매 리뷰에 대한 긍정, 부정의 판별이 가능한 감정 분석 시스템 개발을 진행한다. 

모델의 구현은 하단의 과정을 순서로 개발을 진행한다.

1. 데이터 전처리 (노이즈 제거 및 결측치 처리, 토큰화 및 불용어 제거, 정규화 과정)

2. 모델 구현 

3. 모델 학습 및 튜닝 (dataset을 train, test로 분할, 모델 학습 및 하이퍼파라미터 튜닝(grid search or random search)를 통한 교차 검증)

4. 모델 평가 (정확도, 정밀도, 재현율, F1 score, ROC-AUC 지표를 통한 모델 검증 및 overfitting, underfitting 여부 확인 및 해결 방법)

5. 시각화 및 보고 (모델이 출력한 결과 및 수치값을 matplotlib, Seaborn을 통한 시각화 진행(혼동 행렬 및 ROC 곡선 생성))

-> 상기 내용을 보고서로 작성하여 별도 첨부함.

========================================================================

- Python Version : Python 3.10.13

## 1. 데이터 전처리

In [2]:
# 필요 라이브러리 import

%pip install matplotlib
%pip install pandas

import pandas as pd # pandas 모듈 import
import time # time 모듈 import
import ast # 구문 트리 파싱 및 분석에 사용하는 asr 모듈 import

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


데이터를 우선 살펴보면 "Amazon_uk_shoes_products_dataset_2021_12.csv"는 하단의 순서대로 데이터가 이루어져 있음을 확인 가능하다.

1. url (상품 URL 주소)
2. product_name (상품명)
3. reviewer_name (리뷰 등록자명)
4. review_title (리뷰 제목)
5. review_text (리뷰 내용)
6. review_rating (상품 리뷰 별점)
7. verified_purchase
8. review_date
9. helpful_count
10. uniq_id
11. scraped_at

## 2. 모델 구현

## 3. 모델 학습 및 튜닝

## 4. 모델 평가

## 5. 시각화 및 보고